In [4]:
import pandas as pd

In [5]:
# Load the dataset
data = pd.read_csv('/content/credit_card_default.csv')


In [6]:
# Check for missing values
missing_values = data.isnull().sum()
print("Missing Values:\n", missing_values)

Missing Values:
 ID                            0
LIMIT_BAL                     0
SEX                           0
EDUCATION                     0
MARRIAGE                      0
AGE                           0
PAY_1                         0
PAY_2                         0
PAY_3                         0
PAY_4                         0
PAY_5                         0
PAY_6                         0
BILL_AMT1                     0
BILL_AMT2                     0
BILL_AMT3                     0
BILL_AMT4                     0
BILL_AMT5                     0
BILL_AMT6                     0
PAY_AMT1                      0
PAY_AMT2                      0
PAY_AMT3                      0
PAY_AMT4                      0
PAY_AMT5                      0
PAY_AMT6                      0
default payment next month    0
dtype: int64


In [8]:

# pre-processing on dataset
#  Splitting the Data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [9]:
# Separate features and target variable
X = data.drop(columns=['default payment next month'])  # Features
y = data['default payment next month']  # Target variable

In [10]:
# Define categorical and numerical features
categorical_features = ['SEX', 'EDUCATION', 'MARRIAGE']  # Assuming these are categorical
numerical_features = [col for col in X.columns if col not in categorical_features]

# Define preprocessing steps for numerical and categorical features
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())  # Scale numerical features
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder())  # Encode categorical features
])

# Combine preprocessing steps for numerical and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocess the training data
X_train_preprocessed = preprocessor.fit_transform(X_train)

# Preprocess the testing data
X_test_preprocessed = preprocessor.transform(X_test)

In [12]:
# Perform different algorithms such as kNN,SVM, Gradient Boosting, Ada
#boosting and XG Boost
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Define the classifiers
classifiers = {
    "kNN": KNeighborsClassifier(),
    "SVM": SVC(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "XGBoost": XGBClassifier()
}


In [13]:
# Train and evaluate each classifier
for name, classifier in classifiers.items():
    print(f"Training and evaluating {name}...")
    classifier.fit(X_train_preprocessed, y_train)
    y_pred = classifier.predict(X_test_preprocessed)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy of {name}: {accuracy:.4f}")

Training and evaluating kNN...
Accuracy of kNN: 0.7962
Training and evaluating SVM...
Accuracy of SVM: 0.8185
Training and evaluating Gradient Boosting...
Accuracy of Gradient Boosting: 0.8195
Training and evaluating AdaBoost...
Accuracy of AdaBoost: 0.8170
Training and evaluating XGBoost...
Accuracy of XGBoost: 0.8182


In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Define parameter grids for each classifier
param_grids = {
    "kNN": {'n_neighbors': [3, 5, 7]},
    "SVM": {'C': [0.1, 1, 10], 'gamma': [0.1, 0.01, 0.001], 'kernel': ['rbf']},
    "Gradient Boosting": {'n_estimators': [50, 100, 200], 'learning_rate': [0.1, 0.05, 0.01]},
    "AdaBoost": {'n_estimators': [50, 100, 200], 'learning_rate': [0.1, 0.05, 0.01]},
    "XGBoost": {'n_estimators': [50, 100, 200], 'learning_rate': [0.1, 0.05, 0.01]}
}

# Perform hyperparameter tuning for each classifier
for name, classifier in classifiers.items():
    print(f"Tuning hyperparameters for {name}...")
    param_grid = param_grids[name]

    # Initialize GridSearchCV or RandomizedSearchCV
    if name in ["kNN", "SVM"]:
        search_cv = GridSearchCV(classifier, param_grid, cv=5, scoring='accuracy')
    else:
        search_cv = RandomizedSearchCV(classifier, param_grid, cv=5, n_iter=10, scoring='accuracy')

    # Fit the search_cv to the preprocessed training data
    search_cv.fit(X_train_preprocessed, y_train)

    # Print the best parameters found
    print("Best parameters found:")
    print(search_cv.best_params_)

    # Evaluate the best estimator on the test set
    best_estimator = search_cv.best_estimator_
    y_pred = best_estimator.predict(X_test_preprocessed)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy of {name} after tuning: {accuracy:.4f}")


Tuning hyperparameters for kNN...
Best parameters found:
{'n_neighbors': 7}
Accuracy of kNN after tuning: 0.8012
Tuning hyperparameters for SVM...
